In [1]:
import cv2;
import numpy as np;
classes=[];

In [2]:
def load_model():
    net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
    with open("coco.names",'r') as f:
        classes=f.read().splitlines()
    return net,classes;




def process_image_for_model(frame):
    height,width,channels=frame.shape;
    blob=cv2.dnn.blobFromImage(frame,1/255,(416,416),(0,0,0),swapRB=True,crop=False)
    return blob;
    
    
    

def prediction_by_model(net,frame1,frame,height,width):
    boxes=[];
    confidences=[];
    class_ids=[];
    net.setInput(frame1);
    out_layer_names=net.getUnconnectedOutLayersNames();
    layer_outputs=net.forward(out_layer_names)
    for output in layer_outputs:
        for detection in output:
            scores=detection[5:];
            class_id=np.argmax(scores);
            temp_score=np.max(scores);
            confidence=scores[class_id];
            if confidence>0.1:
                centre_x=int(detection[0]*width);
                centre_y=int(detection[1]*height);
                w=int(detection[2]*width);
                h=int(detection[3]*height);
                x=int(centre_x - w/2);
                y=int(centre_y - h/2);
                boxes.append([x,y,w,h]);
                confidences.append(float(confidence));
                class_ids.append(class_id);
    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.2,0.4);
    font=cv2.FONT_HERSHEY_PLAIN;
    colors=np.random.uniform(0,255,size=(len(boxes),3));
    if len(indexes)>0:
        for i in indexes.flatten():
            x,y,w,h=boxes[i];
            label=str(classes[class_ids[i]]);
            confidence=str(round(confidences[i],2))
            color=colors[i];
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2);
            cv2.putText(frame,label+" "+confidence,(x,y+20),font,1.5,(0,0,0),2);
    return frame;
    
                
                
    
    

def detect_objects(cap,net):
    while True:
        res,frame=cap.read();
        if not res:
            break;
        height,width,channels=frame.shape;
        frame1=process_image_for_model(frame);
        frame=prediction_by_model(net,frame1,frame,height,width);
        cv2.imshow("Predictions",frame);
        k=cv2.waitKey(10);
        if k==ord('q'):
            break;
        
    

In [ ]:
net,classes=load_model();
cap=cv2.VideoCapture(0);
detect_objects(cap,net);
cap.release();
cv2.destroyAllWindows();

In [28]:
cap.release();